In [1]:
!pip install opencv-python
!pip install mediapipe

In [ ]:
import cv2
import mediapipe as mp
import time
from maos.handtracking import HandDetector
%config Completer.use_jedi = False
from IPython.display import clear_output

def euclidean_dist(v1, v2):
    dist = [(v1[i] - v2[i]) ** 2 for i in range(len(v1))]
    return round(sum(dist) ** 0.5, 2)

# Calculate the distance for each position in respect to all the others.
# e.g. for 20 positions, it returns 210 distances (20 + 19 + 18 + ... + 1).
def positions_distances(positions):
    dists = []
    for i in range(len(positions)):
        for j in range(i + 1, len(positions)):
            dists.append(euclidean_dist(positions[i][1:], positions[j][1:]))
    return dists

# The distances below are simply the output of `positions_distances` 
# taken manually by positioning the fingers according to each Libra letter.
A = ("A", [55.01, 105.3, 140.66, 166.03, 113.22, 148.92, 116.1, 98.01, 98.48, 137.35, 95.6, 82.33, 85.87, 122.8, 78.29, 63.51, 81.5, 102.02, 71.34, 59.3, 55.22, 96.18, 126.4, 80.99, 119.62, 83.01, 61.85, 82.02, 120.42, 75.06, 62.03, 88.39, 118.87, 73.38, 62.17, 102.45, 114.06, 84.88, 75.93, 43.05, 75.64, 47.17, 80.06, 47.13, 32.65, 68.62, 94.7, 60.11, 55.76, 92.65, 107.62, 77.03, 75.89, 119.1, 119.85, 100.46, 97.53, 33.24, 39.62, 49.74, 36.88, 44.64, 70.21, 76.49, 64.28, 69.35, 100.84, 100.24, 89.01, 94.89, 131.46, 124.19, 115.0, 116.52, 54.12, 35.74, 51.0, 68.12, 80.28, 68.68, 77.49, 87.21, 110.64, 98.31, 102.79, 112.4, 141.57, 129.0, 127.82, 132.19, 38.63, 3.16, 19.42, 30.59, 47.54, 25.02, 33.11, 61.29, 63.89, 50.36, 58.52, 92.09, 84.77, 76.24, 79.08, 36.69, 57.87, 53.25, 32.98, 53.76, 66.6, 80.78, 63.2, 76.55, 88.87, 110.72, 95.63, 99.32, 105.82, 21.19, 33.38, 47.85, 28.07, 36.25, 64.2, 65.6, 53.46, 61.68, 95.05, 87.36, 79.31, 82.23, 36.07, 64.54, 27.66, 26.48, 62.24, 75.0, 47.8, 51.01, 90.96, 88.68, 73.16, 73.01, 39.85, 8.6, 20.0, 31.0, 39.82, 23.32, 36.07, 62.03, 54.42, 47.54, 52.77, 45.54, 59.23, 57.38, 31.14, 59.36, 73.93, 84.12, 65.8, 76.58, 85.59, 13.93, 36.67, 48.37, 25.5, 35.51, 67.19, 61.74, 51.24, 54.71, 36.89, 58.19, 21.54, 25.71, 64.56, 64.2, 46.69, 47.17, 37.0, 16.28, 26.68, 31.06, 27.59, 19.21, 28.84, 47.01, 61.29, 56.73, 35.78, 54.2, 65.31, 14.87, 43.17, 43.84, 26.0, 29.55, 44.1, 51.97, 25.24, 22.09, 23.71, 18.87, 26.63, 31.4, 43.19, 12.04])
B = ("B", [43.29, 87.71, 109.48, 119.1, 122.04, 162.83, 188.79, 209.7, 120.55, 170.14, 200.0, 222.43, 114.27, 159.81, 186.59, 209.94, 103.48, 136.03, 157.01, 175.1, 50.25, 84.12, 104.12, 99.92, 142.86, 170.07, 192.0, 108.0, 157.26, 187.45, 210.47, 111.09, 154.18, 179.4, 202.16, 111.41, 138.84, 157.43, 173.98, 43.74, 73.01, 60.83, 102.0, 129.03, 151.08, 79.16, 122.58, 151.46, 174.15, 92.96, 127.53, 149.26, 170.26, 106.25, 123.43, 137.3, 150.76, 31.89, 17.12, 59.41, 86.7, 108.78, 38.9, 78.85, 107.79, 130.55, 59.06, 85.76, 106.04, 126.65, 81.02, 88.23, 98.35, 109.86, 20.25, 44.82, 69.77, 90.62, 7.21, 53.15, 83.38, 106.38, 31.26, 54.56, 76.9, 98.86, 58.31, 57.71, 66.48, 78.23, 42.95, 70.21, 92.23, 27.31, 61.85, 90.67, 113.44, 51.2, 70.6, 89.55, 109.77, 77.01, 77.82, 85.15, 95.13, 27.29, 49.37, 46.01, 30.0, 52.0, 73.5, 66.65, 53.85, 60.21, 75.17, 95.27, 79.4, 75.13, 76.56, 22.09, 69.34, 29.41, 31.89, 49.65, 85.63, 57.49, 50.7, 57.63, 112.8, 90.08, 79.25, 74.11, 89.44, 43.14, 26.02, 33.3, 103.08, 67.74, 51.79, 49.04, 128.7, 102.34, 87.73, 78.1, 50.01, 80.23, 103.1, 25.0, 48.47, 71.85, 94.26, 52.95, 50.54, 59.46, 71.69, 30.27, 53.24, 60.22, 28.28, 30.41, 48.1, 85.59, 60.84, 50.25, 47.8, 23.02, 88.1, 46.69, 26.25, 25.81, 110.8, 81.34, 64.35, 52.81, 109.56, 65.86, 40.46, 23.41, 130.51, 99.2, 79.93, 64.82, 45.54, 72.37, 95.71, 28.79, 30.48, 46.49, 62.97, 26.93, 50.22, 64.66, 34.83, 22.02, 24.76, 23.35, 90.79, 58.82, 39.62, 26.57, 113.32, 80.53, 59.91, 43.19, 33.73, 55.32, 73.76, 21.59, 40.02, 18.44])
C = ("C", [48.7, 80.4, 108.12, 137.03, 97.62, 137.97, 157.63, 164.25, 98.08, 146.58, 165.56, 169.1, 96.25, 144.4, 160.64, 165.02, 93.48, 130.92, 146.66, 154.56, 34.93, 62.97, 92.96, 86.68, 122.98, 135.6, 136.49, 93.38, 135.38, 146.17, 142.86, 97.19, 137.93, 144.25, 141.23, 99.05, 128.46, 135.93, 136.86, 28.16, 58.05, 75.54, 104.14, 110.92, 108.0, 85.91, 118.38, 122.77, 115.11, 93.3, 124.15, 123.02, 115.1, 98.23, 118.34, 119.22, 115.11, 30.08, 80.95, 99.25, 99.04, 91.39, 93.15, 114.44, 111.8, 99.16, 102.63, 122.98, 114.44, 101.24, 109.4, 120.81, 115.52, 106.38, 91.93, 97.86, 89.45, 76.32, 104.66, 112.95, 102.4, 84.38, 115.36, 123.98, 107.69, 89.0, 123.43, 125.72, 114.23, 100.02, 40.36, 62.24, 73.98, 12.73, 49.93, 68.51, 76.66, 23.6, 51.24, 63.03, 70.58, 32.25, 43.0, 50.25, 57.2, 27.07, 45.54, 42.95, 15.3, 29.15, 44.55, 50.7, 26.63, 22.67, 36.06, 59.03, 34.0, 16.49, 17.46, 20.62, 67.91, 34.48, 13.0, 17.72, 76.79, 48.37, 19.72, 9.0, 85.48, 59.77, 35.85, 14.21, 82.05, 54.92, 30.53, 8.06, 92.02, 68.8, 39.7, 14.76, 101.08, 79.31, 56.3, 34.83, 48.84, 72.09, 83.73, 11.18, 46.65, 65.0, 76.69, 20.25, 35.17, 48.8, 60.42, 30.27, 51.89, 53.85, 13.89, 20.0, 42.54, 60.84, 27.02, 1.41, 20.62, 24.76, 79.7, 43.42, 10.77, 15.81, 87.8, 57.22, 31.4, 11.7, 93.19, 65.74, 34.83, 9.43, 102.08, 77.49, 53.23, 31.3, 48.8, 71.69, 85.71, 9.22, 34.67, 53.54, 68.01, 32.76, 56.36, 54.2, 16.16, 12.53, 34.44, 25.5, 79.38, 47.01, 21.02, 8.06, 94.43, 68.41, 43.86, 21.93, 38.9, 60.37, 76.16, 26.0, 46.87, 21.93])
D = ("D", [48.01, 90.38, 115.97, 130.65, 133.46, 189.36, 224.15, 253.3, 127.35, 188.86, 170.26, 142.34, 120.07, 178.04, 160.38, 131.75, 114.73, 158.86, 159.52, 142.02, 50.25, 87.85, 114.86, 116.4, 174.43, 210.6, 241.27, 122.92, 184.27, 164.11, 135.13, 126.87, 182.39, 163.25, 134.37, 134.27, 173.52, 171.9, 154.35, 44.0, 79.56, 79.62, 135.09, 171.0, 202.0, 98.11, 153.44, 133.15, 105.85, 113.17, 160.26, 141.16, 115.21, 132.85, 162.69, 158.8, 142.86, 38.59, 37.58, 91.14, 127.0, 158.0, 63.89, 112.0, 92.14, 67.2, 86.76, 123.69, 105.48, 83.82, 114.73, 134.24, 128.81, 115.25, 4.12, 59.91, 95.9, 126.44, 30.08, 74.0, 53.85, 29.0, 58.22, 85.63, 68.01, 50.6, 91.09, 100.72, 94.37, 83.17, 58.05, 94.2, 124.9, 33.73, 74.46, 54.56, 31.24, 62.13, 87.68, 70.43, 54.12, 95.13, 104.0, 97.51, 86.68, 36.22, 67.12, 66.6, 40.8, 35.9, 50.93, 90.21, 73.17, 67.27, 75.72, 122.8, 108.34, 100.13, 99.3, 31.0, 98.49, 49.24, 58.8, 82.76, 117.69, 83.01, 85.45, 102.83, 147.32, 123.0, 114.93, 119.28, 126.46, 69.81, 84.53, 111.07, 142.58, 98.86, 106.3, 127.95, 169.53, 139.2, 131.65, 139.3, 62.2, 42.95, 15.81, 29.15, 62.18, 43.05, 20.62, 63.15, 71.12, 65.19, 53.14, 20.4, 49.16, 73.36, 34.71, 36.8, 59.09, 99.82, 73.93, 65.8, 70.43, 29.0, 58.9, 40.16, 31.4, 44.05, 89.2, 72.56, 64.38, 63.69, 41.23, 56.64, 39.2, 28.44, 75.27, 74.73, 67.68, 59.21, 58.0, 40.61, 14.87, 34.21, 49.52, 46.04, 29.7, 19.31, 48.05, 74.69, 40.5, 32.8, 42.54, 29.15, 63.25, 41.19, 33.06, 33.84, 47.42, 50.61, 45.0, 32.57, 45.01, 47.52, 32.31, 8.25, 19.85, 17.72])
E = ("E", [46.17, 92.48, 120.43, 120.42, 130.25, 179.77, 173.99, 152.59, 125.69, 185.72, 172.7, 143.89, 118.07, 175.07, 162.44, 133.73, 113.51, 156.52, 154.3, 134.84, 50.0, 84.02, 95.21, 111.31, 163.62, 155.72, 132.77, 117.61, 178.23, 163.01, 133.02, 121.49, 175.85, 161.62, 132.91, 129.27, 167.26, 162.96, 143.6, 37.95, 62.17, 83.24, 133.09, 123.49, 100.84, 101.36, 155.9, 139.47, 111.23, 117.39, 162.61, 147.85, 122.41, 137.01, 165.52, 159.36, 142.13, 34.66, 56.22, 100.28, 89.99, 68.71, 81.84, 127.53, 111.02, 86.05, 105.55, 140.26, 125.94, 105.0, 132.2, 151.43, 144.27, 129.87, 21.95, 71.18, 62.01, 39.01, 47.71, 94.54, 78.0, 51.61, 73.68, 105.65, 91.29, 71.02, 103.1, 117.83, 110.45, 97.17, 52.35, 44.64, 22.36, 29.83, 72.8, 56.3, 29.83, 59.14, 84.17, 70.04, 52.55, 91.09, 99.4, 91.71, 80.52, 10.82, 32.25, 57.01, 35.78, 25.02, 38.29, 80.8, 64.63, 57.14, 64.14, 112.0, 97.74, 89.94, 89.89, 23.09, 54.56, 45.49, 32.57, 36.89, 81.01, 72.28, 63.2, 65.55, 113.22, 103.04, 95.08, 92.89, 41.11, 60.0, 43.93, 29.15, 70.83, 79.4, 67.12, 59.01, 103.81, 102.96, 94.92, 88.0, 61.07, 47.07, 18.87, 30.15, 61.29, 46.53, 23.32, 62.97, 70.24, 62.77, 50.7, 16.55, 45.54, 72.01, 32.76, 32.02, 54.13, 97.0, 70.77, 63.95, 70.77, 30.0, 63.25, 39.81, 32.39, 45.34, 91.83, 73.16, 65.51, 67.6, 44.28, 54.45, 40.61, 30.27, 76.9, 73.98, 65.97, 59.08, 57.01, 44.72, 18.03, 32.98, 48.17, 42.44, 24.84, 14.76, 42.95, 72.9, 39.12, 33.3, 45.62, 29.07, 65.6, 40.79, 33.24, 38.9, 47.89, 46.96, 39.45, 29.02, 44.0, 43.74, 27.29, 8.06, 23.85, 19.65])
alphabet = [A, B, C, D, E]

cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

while True:
    clear_output(wait=True)
    _, img = cap.read()
    detector = HandDetector()
    img = detector.find_hands(img)
    positions = detector.find_position(img, 0)
    
    # Very naive implementation:
    # Get the distances of the current frame and compare to the distances
    # of each letter in the alphabet, and display the one with the lowest distance.
    if len(positions) > 0:
        points_distances = positions_distances(positions)
        dists = []
        for letter, distances in alphabet:
            dists.append((letter, euclidean_dist(points_distances, distances)))

        letter = sorted(dists, key=lambda i: i[1])[0][0]
        cv2.rectangle(img, (10, 10), (50, 50), (255, 255, 255), cv2.FILLED)
        cv2.putText(img, str(letter), (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (100, 100, 100), 2, cv2.LINE_AA)
    
    cv2.imshow("Video", img)
    cv2.waitKey(1)